In [92]:
import torch
import torch.nn as nn
import torch.nn.functional as Fn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class Model(nn.Module):
    def __init__(self, in_features=2, hidden1=4, hidden2=3, out_features=1):
        super().__init__()
        self.Layer_1 = nn.Linear(in_features, hidden1)
        self.Layer_2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)

    def forward(self, x):
        x = Fn.relu(self.Layer_1(x))
        x = Fn.relu(self.Layer_2(x))
        x = torch.sigmoid(self.out(x))

        return x

In [ ]:
model = Model()

data = load_breast_cancer()
X, y = data.data, data.target

X_selected = X[:, [0, 1]]

plt.scatter(X_selected[y == 0, 0], X_selected[y == 0, 1], color='blue', label='Benign', alpha=0.5)
plt.scatter(X_selected[y == 1, 0], X_selected[y == 1, 1], color='red', label='Malignant', alpha=0.5)
plt.xlabel('Mean Radius')
plt.ylabel('Mean Texture')
plt.legend()
plt.show()

#Linear model can't work with this data

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

torch.manual_seed(42)

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

epochs = 16
epoch_count = []
loss_values = []
test_loss_values = []